# Chapter 9: Population Methods

In [3]:
import numpy as np
from scipy import stats
from copy import copy

## Algorithm 9.1

In [78]:
def rand_population_uniform(m, a, b):
    d = a.shape[0]
    return [ a + np.random.random(d)*(b-a) for i in range(m) ]

## Algorithm 9.2

In [11]:
def rand_population_normal(m ,mu, sigma):
    D = stats.multivariate_normal(mean=mu, cov=sigma)
    return [ D.rvs() for i in range(m)]

## Algorithm 9.3

In [13]:
def rand_population_cauchy(m, mu, sigma):
    n = mu.shape[0]
    return [ [stats.cauchy(loc=mu[i],scale=sigma[i]).rvs() for i in range(n)] for j in range(m)]

## Algorithm 9.4

In [ ]:
def genetic_algorithm(f, population, k_max, S, C, M):
    for k in range(k_max):
        y = [ f(pop_val)  for pop_val in population]
        parents = S.select(y)
        children = [C.crossover( population[p[0]], population[p[1]] ) for  p in parents]

        population = [M.mutate(child) for child in children]
    return population[ np.argmin(f(population))]

### Example

In [ ]:
f = lambda x : np.linalg.norm(x)
m = 100
k_max = 10
population = rand_population_uniform(m, np.array([-3, 3]), np.array([3,3]) )
S = TruncationSelection(10)
C = SinglePointCrossover()
M = GaussianMutation(0.5)

x = genetic_algorithm(f, population, k_max, S, C, M)

## Algorithm 9.5

In [40]:
def rand_population_binary(m, n):
    nums = []
    for i in range(m):
        arr = np.random.randint(2, size=(n,))
        num = str(arr).replace('[','').replace(']','').replace(' ','')
        nums.append(num)
    return nums

## Algorithm 9.6

In [41]:
class TruncationSelection:

    def __init__(self,k):
        self.k = k

    def select(self, y):
        p = np.argsort(y)
        return [ p[np.random.randint(self.k, size=(2,))] for i in y ]


class TournamentSelection:

    def __init__(self,k):
        self.k = k

    def select(self, y):
        return [ [self.getparent(y), self.getparent(y)] for i in y ]

    def getparent(self, y):
        p = np.random.permutation(len(y))
        y_np = np.array(y)
        return np.argmin(y_np[p[:self.k]])

class RouletteWheelSelection:

    def select(self):
        y = np.max(y) - np.array(y)
        normalize = y / np.linalg.norm(y, 1)
        return [np.random.choice(a=np.arange(0,len(y),1), p=normalize, size=2 ) for i in y]


## Algorithm 9.7

In [93]:
'''
For array of numbers
'''
class SinglePointCrossover:
    
    
    def crossover(self,a,b):
        i = np.random.randint(low=0, high=len(a))
        return np.concatenate((a[:i], b[i:]),axis=0)

class TwoPointCrossover:
    
    
    def crossover(self, a, b):
        n = len(a)
        i, j = np.random.randint(low=0,high=n,size=2)

        if i > j:
            (i, j) = (j,i)
        return np.concatenate((np.concatenate((a[:i], b[i:j]),axis=0),a[j:n]),axis=0)

class UniformCrossover:
    
    def crossover(self, a, b):
        child = copy(a)
        for i in range(len(a)):
            if np.random.random() < 0.5:
                child[i] = b[i]
        return child

## Algorithm 9.8

In [ ]:
def interpolationCrossover(lambda_, a, b):
    return np.array(a)*(1-lambda_) + np.array(b)*lambda_

## Algorithm 9.9

In [15]:
class BitwiseMutation:

    def __init__(self, lambda_):
        self.lambda_= lambda_

    def mutate(self, child):
        return [ int(not v) if np.random.random() < self.lambda_ else v for v in child ]

class GaussianMutation:

    def __init__(self, sigma):
        self.sigma= sigma

    def mutate(self, child):
        return child + np.random.normal(size=len(child))*self.sigma